# requests-html

In [13]:
# !pip install requests-html 

In [3]:
url = (
  "https://pt.wikipedia.org/wiki/Lista_de_parlamentares_do_Rio_de_Janeiro"
)

In [4]:
# requests_html: This library intends to make parsing HTML 
# (e.g. scraping the web) as simple and intuitive as possible.
from requests_html import HTMLSession
session = HTMLSession()

In [5]:
r = session.get(url)

In [6]:
# Select Elements within Elements
r.html.find("table", first=True)
# r.html.find("table", first=True).text
# r.html.find('a', containing='kenneth')

<Element 'table' class=('wikitable',) style='font-size: 85%;'>

In [7]:
# Search for text on the page:
r.html.search('({})')
# r.html.search('({ano})')

<Result ('1891–1960',) {}>

In [8]:
# Select an Element with a CSS Selector
r.html.find('#mw-content-text')
# r.html.find('.mw-content-text')

[<Element 'div' id='mw-content-text' class=('mw-body-content', 'mw-content-ltr') lang='pt' dir='ltr'>]

In [12]:
tables = r.html.find("table")

table_df = tables[1]
table_df

<Element 'table' class=('wikitable',) style='font-size: 85%;'>

In [13]:
rows = table_df.find("tr")
rows

[<Element 'tr' >,
 <Element 'tr' >,
 <Element 'tr' >,
 <Element 'tr' >,
 <Element 'tr' >,
 <Element 'tr' >,
 <Element 'tr' >]

In [16]:
headers = rows[0].find("th")
headers[0].text

'Senadores eleitos'

In [18]:
headers_list = []

for header in headers:
    headers_list.append(header.text)

In [ ]:
dados = []

for row in rows[1:]:
    itens = row.find("td")

    r = []
    
    for item in itens:
        r.append(item.text)

        dados.append(r)

In [1]:
import pandas as pd

/Users/gabriela/Development/Contrib/okb/python-inovacao-civica/notebooks/venv/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [ ]:
pd.DataFrame(dados, columns=headers)

# pandas

In [1]:
import pandas as pd

url = (
  "https://pt.wikipedia.org/wiki/Lista_de_parlamentares_do_Rio_de_Janeiro"
)

tables = pd.read_html(url)

/Users/gabriela/Development/Contrib/okb/python-inovacao-civica/notebooks/venv/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
print(f'Total tables: {len(tables)}')

Total tables: 9


In [3]:
tables[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Senadores eleitos  8 non-null      object
 1   Naturalidade       8 non-null      object
 2   Mandatos           8 non-null      int64 
 3   Ano da eleição     8 non-null      int64 
dtypes: int64(2), object(2)
memory usage: 384.0+ bytes


In [11]:
tables[0].head()

,Senadores eleitos,Naturalidade,Mandatos,Ano da eleição
0,Hamilton Nogueira,"Campos dos Goytacazes, RJ",1,1945
1,Luís Carlos Prestes[nota 1],"Porto Alegre, RS",1,1945
2,Mário Ramos,"Rio de Janeiro, RJ",1,1947
3,Alencastro Guimarães,"São Sebastião do Caí, RS",1,1950
4,Mozart Lago[nota 2],"Nova Friburgo, RJ",1,1950


In [12]:
tables[1].head()

,Senadores eleitos,Naturalidade,Mandatos,Ano da eleição
0,Danton Jobim[nota 5][nota 6],"Avaré, SP",2,"1970, 1974"
1,Gilberto Marinho,"Pelotas, RS",1,1962
2,Aurélio Viana,"Maceió, AL",1,1962
3,Mário Martins[nota 5],"Petrópolis, RJ",1,1966
4,Benjamin Farah[nota 6],"Corumbá, MS",1,1970


In [10]:
# !pip install html5lib

In [5]:
# tables_match = pd.read_html(url, match='Estado da Guanabara')
# len(tables_match)